# Quantum Key Distribution (QKD): 
## BB84 protocol implementation

Developed in 1984 by Charles Bennett and Gilles Brassard, the BB84 protocol is one of the earliest Quantum Key Distribution (QKD) protocols. Its enables the secure establishment of a secret key through a quantum channel between two parties, Alice and Bob, leveraging the uncertainty principle and the no-cloning theorem from quantum mechanics. This prevents any potential eavesdropper, such as Eve, from intercepting the key without detection. 

### Install qiskit packages

In [1]:
%pip install qiskit-ibm-provider
%pip install qiskit

     |████████████████████████████████| 245 kB 2.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute,IBMQ
from qiskit_ibm_provider import IBMProvider

In [2]:
api_token = "b329ccea0ba6aeffaa2633b9490c4077b92d9cd796ccb6f82d8c63d0e34199359b5feed830213b466f0e8703cc82bd39c8c619a08ff43c41467159813542acb9"

### Set up IBM account

In [3]:
IBMQ.enable_account(api_token)
provider = IBMQ.get_provider(hub='ibm-q')
print(provider.backends())
backend = provider.get_backend('ibmq_qasm_simulator')


/tmp/ipykernel_1057634/1614407315.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.enable_account(api_token)
/tmp/ipykernel_1057634/1614407315.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.enable_account(api_token)


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_brisbane') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_kyoto') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_osaka') from IBMQ(hub='ibm-q', group='open', project='main')>]


### Set up quantum circuit

In [12]:
#Set up quantum circuit
def set_up_qc(qubits):
    q = QuantumRegister(qubits, 'q')
    c = ClassicalRegister(qubits, 'c')
    qc = QuantumCircuit(q, c)
    return qc, q, c

In [13]:
qubits = 7
qc, q, c = set_up_qc(qubits)

### Alice generates a random key

In [14]:
def generate_random_bits(qc, q, c):
    qc.h(q)
    qc.measure(q, c)
    job = execute(qc, backend, shots=1)
    # job_monitor(job)
    counts = job.result().get_counts()
    key = list(counts.keys())[0]
    return key

In [15]:
alice_key = generate_random_bits(qc, q, c)
print("Alice Key:", alice_key)

Alice Key: 0001110


### Alice generates random bases to encode the key

In [17]:
alice_bases = generate_random_bits(qc, q, c)
print("Alice Bases:", alice_bases)

Alice Bases: 0010100


### Alice encodes key using bases
```
example:
01000100 - basis
01011010 - key
0?011?10 - result
```
```
basis 1 = Hadamard Basis (Diagonal)
basis 0 = Computational Basis (Recilinear)
```

In [18]:
def encode(key, bases, qubits):
    qc = QuantumCircuit(q,c)
    encoded_key = ""
    gates = ""
    for i in range(qubits):
        #basis 1 = Hadamard Basis
        #basis 0 = Computational Basis
        if key[i] == '1':
                qc.x(i)
        if bases[i] == '1':
            qc.h(i)
            gates += "H"
            if key[i] == '1':
                encoded_key += "|-⟩"
            else:
                encoded_key += "|+⟩"
        else:
            if key[i] == '1':
                gates += "x"
                encoded_key += "|1⟩"
            else:
                gates += "."
                encoded_key += "|0⟩"
    return encoded_key, gates, qc

In [19]:
alice_encoded_key, alice_gates, alice_qc = encode(alice_key, alice_bases, qubits)
print("Alice Encoded Key:", alice_encoded_key)   
print("Alice Gates Applied:", alice_gates)  

Alice Encoded Key: |0⟩|0⟩|+⟩|1⟩|-⟩|1⟩|0⟩
Alice Gates Applied: ..HxHx.


### Bob generates random bases

In [21]:
bob_bases = generate_random_bits(qc, q, c)
print("Bob Bases:", bob_bases)

Bob Bases: 0111100


### Bob decodes Alice's encoded key using Bob's bases

* test using alice bases (should return alice's original key)

In [22]:
def decode(qc, bases, qubits):
    gates = ""
    for i in range(qubits):
        if bases[i] == '1':
        # if alice_bases[i] == '1': //test using alice bases (should return alice's original key)
            qc.h(i)
            gates += "H"
        else:
            gates += "."
    qc.measure_all()
    job = execute(qc, backend, shots=1)
    counts = job.result().get_counts()
    decoded_key = list(counts.keys())[0][0:qubits][::-1]
    return decoded_key, gates
    

In [23]:
bob_decoded_key, bob_gates = decode(alice_qc, bob_bases, qubits)
print("Bob Decoded Key:", bob_decoded_key)    
print("Bob Gates Applied:", bob_gates)

Bob Decoded Key: 0101110
Bob Gates Applied: .HHHH..


### Alice shares encoding bases classically and Bob compares bases with Alice

Keys won't match due to noise or iterception by Eve. If keys don't match then throw them away and try again.

In [24]:
def find_shared_key(alice_bases, bob_bases, qubits):
    alice_shared_key = ""
    bob_shared_key = ""
    for i in range(qubits):
        if alice_bases[i] == bob_bases[i]:
            alice_shared_key += alice_key[i]
            bob_shared_key += bob_decoded_key[i]
    return alice_shared_key, bob_shared_key
    
        

In [25]:
alice_shared_key, bob_shared_key = find_shared_key(alice_bases, bob_bases, qubits)
print("Alice Shared Key:", alice_shared_key)
print("Bob Shared Key:", bob_shared_key)
if alice_shared_key == bob_shared_key:
    print("Keys match!")
else:
    print("Keys don't match due to noise or interception by Eve")
    print("Discard the keys and run algorithm again")

Alice Shared Key: 00110
Bob Shared Key: 00110
Keys match!


### Using more qubits

In [40]:
qubits = 256
qc, q, c = set_up_qc(qubits)
print(qubits, "qubit key")
print()

alice_key = generate_random_bits(qc, q, c)
alice_bases = generate_random_bits(qc, q, c)

alice_encoded_key, alice_gates, alice_qc = encode(alice_key, alice_bases, qubits)

print("Alice key:", alice_key)
print("Alice bases:", alice_bases)
print("Alice encoded key:", alice_encoded_key)
print()

bob_bases = generate_random_bits(qc, q, c)
bob_decoded_key, bob_gates = decode(alice_qc, bob_bases, qubits)
print("Bob bases:", bob_bases)
print("Bob decoded key:", bob_decoded_key)
print()

alice_shared_key, bob_shared_key = find_shared_key(alice_bases, bob_bases, qubits)
print("Alice shared key:", alice_shared_key)
print("Bob shared key:", bob_shared_key)
print(f'{len(bob_shared_key)} qubit key, {(len(bob_shared_key)/qubits)*100:.2f}% of initial key')
print()
print("Alice and Bob shared key match?:", alice_shared_key==bob_shared_key)

256 qubit key

Alice key:	 1111000111010100111010100011010101100001010111011101111000000000100001001110101101001001110101010100101011101010000100100011100101010001011110100000001010011100110000010100101101100001101110111001110011000111001100110001110101000000110101011110110100100110
Alice bases:	 0011100010110001110110101100111000001000011011010001100000110001000110100001111100010000010100011100001001111000001111000110010011011100001000100011111000111001101101100011100100001001000010000111110010100000101101101000000001011000011000011110011111000101
Alice encoded key: |1⟩|1⟩|-⟩|-⟩|+⟩|0⟩|0⟩|1⟩|-⟩|1⟩|+⟩|-⟩|0⟩|1⟩|0⟩|+⟩|-⟩|-⟩|1⟩|+⟩|-⟩|0⟩|-⟩|0⟩|+⟩|+⟩|1⟩|1⟩|+⟩|-⟩|+⟩|1⟩|0⟩|1⟩|1⟩|0⟩|+⟩|0⟩|0⟩|1⟩|0⟩|-⟩|+⟩|1⟩|-⟩|-⟩|0⟩|-⟩|1⟩|1⟩|0⟩|-⟩|-⟩|1⟩|1⟩|0⟩|0⟩|0⟩|+⟩|+⟩|0⟩|0⟩|0⟩|+⟩|1⟩|0⟩|0⟩|+⟩|+⟩|1⟩|+⟩|0⟩|1⟩|1⟩|1⟩|+⟩|-⟩|+⟩|-⟩|-⟩|0⟩|1⟩|0⟩|+⟩|1⟩|0⟩|0⟩|1⟩|1⟩|-⟩|0⟩|-⟩|0⟩|1⟩|0⟩|-⟩|+⟩|-⟩|0⟩|0⟩|1⟩|0⟩|-⟩|0⟩|1⟩|-⟩|-⟩|+⟩|-⟩|0⟩|1⟩|0⟩|0⟩|0⟩|+⟩|-⟩|+⟩|+⟩|1⟩|0⟩|0⟩|+⟩|-⟩|1⟩|1⟩|+⟩|0⟩|1⟩|+⟩|-⟩|0⟩|-⟩|+⟩|+⟩|0⟩|1⟩|0⟩|1⟩|-⟩|1⟩|1⟩|0⟩